In [3]:
using GradientRobustMultiPhysics
using ExtendableGrids
using GridVisualize

const g = DataFunction([0,10]; name = "g")

verbosity = 0
E = 1000
ν = 0.4
Plotter = nothing



In [7]:
xgrid = simplexgrid("../assets/2d_grid_cookmembrane.sg")
xgrid = uniform_refine(xgrid,2)

## compute Lame' coefficients from E and ν
μ = (1/(1+ν))*E
λ = (ν/(1-2*ν))*μ

## PDE description via prototype and add data
Problem = LinearElasticityProblem(2; shear_modulus = μ, lambda = λ)
add_rhsdata!(Problem, 1, LinearForm(Identity, g; regions = [2], AT = ON_BFACES))
add_boundarydata!(Problem, 1, [4], HomogeneousDirichletBoundary)

## show and solve PDE
@show Problem
FEType = H1P1{2} # P1-Courant FEM will be used
Solution = FEVector("u_h",FESpace{FEType}(xgrid))
solve!(Solution, Problem)

    ## plot stress on displaced mesh
displace_mesh!(xgrid, Solution[1]; magnify = 4)
p = GridVisualizer(; Plotter = Plotter, layout = (1,2), clear = true, resolution = (1000,500))
scalarplot!(p[1,1], xgrid, view(nodevalues(Solution[1]; abs = true),1,:), levels = 7, title = "u_h")
vectorplot!(p[1,1], xgrid, evaluate(PointEvaluator(Solution[1], Identity)), spacing = 5, clear = false, title = "u_h (abs + quiver)")
scalarplot!(p[1,2], xgrid, view(nodevalues(Solution[1], SymmetricGradient{1/√2}; abs = true),1,:), levels = 7, title = "ϵ(u_h) (abs)")


Problem = 
PDE-DESCRIPTION
  system name = linear elasticity problem

      id    | unknown name / variables [#A, #T] / equation name
     [1]    | u / ["u", "v"] / displacement equation 

  LHS block | PDEOperator(s)
    [1,1]   | (ℂ(μ,λ) ϵ(u),ϵ(v)) (APT = BilinearForm, AT = ON_CELLS, regions = [0])

  RHS block | PDEOperator(s)
     [1]    | (g, id(v)) (APT = LinearForm, AT = ON_BFACES, regions = [2])

   BoundaryOperators[1] : HomogeneousDirichletBoundary (bregions = [4])





┌ Info: ========== Solving linear elasticity problem ==========
└ 	Equation (1.1) displacement equation for u (discretised by H1P1{2}, ndofs = 3738)
[ Info: overall residual = 5.417651094778756e-11
┌ Warning: residual was larger than desired target_residual = 1.0e-12!
└ @ GradientRobustMultiPhysics ~/OneDrive/ResearchWork/codes/JuliaFE_Julia23/src/solvers.jl:1388
